In [19]:
%load_ext autoreload
%autoreload 2

In [4]:
import torch
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print('GPU acceleration is available and will be used :-)')
else:
    DEVICE = torch.device("cpu")
    print('GPU is not available. If you are using Google Colab, change the runtime to GPU, otherwise training will '
          'take too long.')

GPU is not available. If you are using Google Colab, change the runtime to GPU, otherwise training will take too long.


In [75]:
!sh download_data.sh

--2021-01-27 21:16:43--  https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.251.30
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.251.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209850483 (200M) [application/json]
Saving to: ‘./data/personachat_self_original.json.1’

personachat_self_or 100%[===================>] 200.13M  5.70MB/s    in 36s     

2021-01-27 21:17:19 (5.60 MB/s) - ‘./data/personachat_self_original.json.1’ saved [209850483/209850483]



In [65]:
from data_util import generate_train_validation_test_files

generate_train_validation_test_files()

0 lines removed
Train set 131,438
0 lines removed
Test set 7,801


In [66]:
from data_util import DataWrapper

# Dataset objects
dw = DataWrapper()
train_ds, val_ds, test_ds = dw.get_datasets(
    train_size=99,
    val_size=99,
    use_glove=True
)
print(f'Train set size: {len(train_ds):,}')
print(f'Validation set size: {len(validation_ds):,}')
print('Vocab size: ', dw.vocab_size)

Train set size: 99
Validation set size: 999
Vocab size:  500


In [67]:
# DataLoader objects
train_iter, val_iter, test_iter = dw.get_dataloaders(
    train_ds, val_ds, test_ds,
    batch_size=2400,
    device=DEVICE
)

x = next(iter(train_iter))
print('Example \n-------')
print(x.src[0])
print(x.src[1])

Example 
-------
tensor([[  2,  21,  14,  ...,   1,   1,   1],
        [  2, 130,  10,  ...,   5,   3,   1],
        [  2,  34,  21,  ...,  61, 154,   3],
        ...,
        [  2,  63,  10,  ...,   1,   1,   1],
        [  2,  34,  21,  ...,   1,   1,   1],
        [  2,  63,   5,  ...,   1,   1,   1]])
tensor([50, 62, 63, 62, 55, 55, 52, 51, 51, 48, 48, 44, 43, 42, 39, 34, 25, 33,
        33, 32, 30, 29, 26, 29, 28, 23, 19, 16,  7, 16, 10, 10, 10, 11,  9,  8,
         9,  7,  8])


In [68]:
%autoreload 2
from model import Seq2seqRNN

vocab_size = dw.vocab_size
embedding_dim = dw.embedding_dim
hidden_dim = 256
n_layers = 3
n_directions_encoder = 2
model = Seq2seqRNN(vocab_size,
                   embedding_dim,
                   hidden_dim,
                   n_layers,
                   n_directions_encoder,
                   dropout=0.2,
                   pretrained_embeddings=dw.embeddings,
                   freeze_embeddings=False)

In [69]:
%autoreload 2
from model import count_parameters

print(f'The model has {count_parameters(model):,} parameters')

The model has 5,484,868 parameters


In [72]:
from train import Seq2seqRNNTrainer

trainer = Seq2seqRNNTrainer(model,
                            train_iter,
                            val_iter,
                            learning_rate=3e-4,
                            pad_token_id=dw.pad_token_id,
                            gradient_clip=99999,
                            teacher_forcing=0.5)

In [74]:
n_epochs = 10
for epoch in range(n_epochs):
    trainer.train(epoch)
    trainer.validate(epoch)

  0%|          | 0/99 [00:00<?, ?it/s]

Epoch 0: train | Loss:   5.978 | Perplexity: 394.618


  0%|          | 0/99 [00:00<?, ?it/s]

Epoch 0: test | Loss:   5.479 | Perplexity: 239.707


  0%|          | 0/99 [00:00<?, ?it/s]

...

Epoch 8: test | Loss:   4.160 | Perplexity:  64.084


  0%|          | 0/99 [00:00<?, ?it/s]

Epoch 9: train | Loss:   4.635 | Perplexity: 103.048


  0%|          | 0/99 [00:00<?, ?it/s]

Epoch 9: test | Loss:   4.156 | Perplexity:  63.832
